In [1]:
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from scipy.stats import pearsonr
from xgboost import XGBRegressor
from sklearn.base import clone
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import gc

warnings.filterwarnings("ignore")
import os
os.chdir("/Users/deweywang/Desktop/GitHub/FlightRank/")


In [10]:
import pandas as pd
import os

model_dir = "model_output/all_features_xgb"
label = "xlarge"
model_importance_dir = os.path.join(model_dir, "model_importance")
csv_path = os.path.join(model_importance_dir, f"feature_importance_{label}_all_features.csv")

# 讀取
df = pd.read_csv(csv_path)

# 第幾名開始沒用過
idx_first_unused = df[df["min_rank"] == 9999].index.min()

print(f"\n✅ 第 {idx_first_unused} 名後都是完全未使用的特徵\n")

# 看第一個
print("✅ 第一個未使用特徵：")
print(df.iloc[idx_first_unused])


features = (df.iloc[:idx_first_unused]['feature'].tolist())



✅ 第 78 名後都是完全未使用的特徵

✅ 第一個未使用特徵：
feature            legs1_segments3_marketingCarrier_code
weight                                               NaN
weight_rank_pos                                   9999.0
gain                                                 NaN
gain_rank_pos                                     9999.0
cover                                                NaN
cover_rank_pos                                    9999.0
min_rank                                          9999.0
Name: 78, dtype: object


In [2]:
import polars as pl

def split_data_by_group_size_per_label_features(
    df: pl.DataFrame,
    bins: list,
    labels: list,
    label_features: dict = None
):
    """
    一次切分所有分群，每個分群用各自features。
    
    df: polars.DataFrame
    bins: 分群邊界
    labels: 分群名稱
    label_features: dict(label -> feature list)，每個label自己的feature欄位。
                   如果為None，則所有分群都用df的所有columns。
    """
    df = df.with_row_count("global_row_nr")

    group_counts = (
        df.group_by("ranker_id")
          .agg(pl.count().alias("n_rows"))
          .filter(pl.col("n_rows") >= bins[0])
    )

    bins_fixed = bins.copy()
    if bins_fixed[-1] is None:
        max_value = group_counts["n_rows"].max()
        bins_fixed[-1] = int(max_value) + 1

    if len(labels) != len(bins_fixed) - 1:
        raise ValueError(f"bins={bins_fixed} 有 {len(bins_fixed)-1}個區間，但labels數={len(labels)}")

    cond = (
        pl.when((pl.col("n_rows") >= bins_fixed[0]) & (pl.col("n_rows") < bins_fixed[1]))
        .then(pl.lit(labels[0]))
    )
    for i in range(1, len(labels)):
        cond = cond.when(
            (pl.col("n_rows") >= bins_fixed[i]) & (pl.col("n_rows") < bins_fixed[i+1])
        ).then(pl.lit(labels[i]))
    cond = cond.otherwise(pl.lit("unknown"))

    group_counts = group_counts.with_columns([
        cond.alias("group_category")
    ])

    df = df.join(group_counts, on="ranker_id", how="left")

    split_data = {}
    for lbl in labels:
        subset = df.filter(pl.col("group_category") == lbl)

        # 如果沒給 label_features，就用全部 columns
        if label_features is None:
            feats = [c for c in df.columns if c not in ("group_category")]
        else:
            feats = label_features.get(lbl, [])

        base_cols = ["selected", "ranker_id", "global_row_nr", "group_category"]
        all_cols = feats + base_cols
        all_cols = list(dict.fromkeys(all_cols))  # 去重防止重複

        subset = subset.select([c for c in all_cols if c in subset.columns])

        mem_mb = subset.estimated_size() / (1024 * 1024)
        print(f"✅ {lbl}: {subset.height} rows, approx {mem_mb:.2f} MB")
        split_data[lbl] = subset

    summary = (
        group_counts.group_by("group_category")
        .agg([
            pl.count().alias("n_groups"),
            pl.col("n_rows").sum().alias("total_rows"),
            pl.col("n_rows").mean().alias("avg_rows_per_group")
        ])
        .sort("group_category")
    )

    print("✅ 分群統計：")
    print(summary)

    return {
        "data_all": df,
        "split_data": split_data,
        "summary": summary
    }


In [3]:
import os
import pandas as pd

def load_label_features(
    model_dir: str,
    split_labels: list,
    top_n: int = None
):
    """
    從模型資料夾讀取每個分群的特徵重要性檔案，回傳每個label的features list。
    
    參數:
    - model_dir: 模型資料夾
    - split_labels: 分群名稱list
    - top_n: 如果指定，只取前N個特徵；否則用第一個min_rank=9999為止

    回傳:
    - dict(label -> features list)
    """
    label_features = {}

    for label in split_labels:
        model_importance_dir = os.path.join(model_dir, "model_importance")
        csv_path = os.path.join(model_importance_dir, f"feature_importance_{label}_all_features.csv")

        df = pd.read_csv(csv_path)

        if top_n is not None:
            # 直接取前top_n
            feats = df.iloc[:top_n]["feature"].tolist()
            print(f"\n✅ {label}: 取前 {top_n} 個特徵")
        else:
            idx_first_unused = df[df["min_rank"] == 9999].index.min()
            feats = df.iloc[:idx_first_unused]["feature"].tolist()
            print(f"\n✅ {label}: 第 {idx_first_unused} 名後都是完全未使用的特徵")
            print("✅ 第一個未使用特徵：")
            print(df.iloc[idx_first_unused])

        label_features[label] = feats

    # 印出所有分群features數量
    for label in split_labels:
        print(f"{label}: {len(label_features[label])} features")

    return label_features


In [ ]:
import numpy as np
import polars as pl
import pandas as pd
import os

model_dir = "model_output/all_features_xgb"

# 先把每個分群都讀一遍feature importance
split_labels = ["small", "medium", "large", 'xlarge']

label_features = load_label_features(
    model_dir=model_dir,
    split_labels=split_labels)

# 讀 train data
train_filled = pl.read_parquet("data/train_filled.parquet")

# 呼叫新版本分群切分
result = split_data_by_group_size_per_label_features(
    df=train_filled,
    bins=[3, 27, 162, 1000, None],
    labels=split_labels,
    label_features=label_features
)

# 清理記憶體
import gc
del train_filled
gc.collect()



✅ small: 第 95 名後都是完全未使用的特徵
✅ 第一個未使用特徵：
feature            legs0_all_segments_carrier_same
weight                                         NaN
weight_rank_pos                             9999.0
gain                                           NaN
gain_rank_pos                               9999.0
cover                                          NaN
cover_rank_pos                              9999.0
min_rank                                    9999.0
Name: 95, dtype: object

✅ medium: 第 99 名後都是完全未使用的特徵
✅ 第一個未使用特徵：
feature            legs1_segments0_aircraft_code_is_missing
weight                                                  NaN
weight_rank_pos                                      9999.0
gain                                                    NaN
gain_rank_pos                                        9999.0
cover                                                   NaN
cover_rank_pos                                       9999.0
min_rank                                             9999.0
Name: 9

0

: 

In [33]:
import os
import gc
import xgboost as xgb
import numpy as np
import polars as pl
from scripts.group_wise import prepare_train_val_split
from scripts.hitrate import compute_hitrate_at_3

# 自訂模型儲存目錄
model_dir = "model_output/selected_features_xgb/v2_top20/"
os.makedirs(model_dir, exist_ok=True)
print(f"✅ Model folder: {model_dir}")

# 預設 XGB 參數
xgb_params = {
    'objective': 'rank:pairwise',
    'eval_metric': 'ndcg@3',
    'max_depth': 10,
    'min_child_weight': 10,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'lambda': 10.0,
    'learning_rate': 0.05,
    'seed': 42,
    'n_jobs': -1,
}

split_labels = ["small", "medium", "large"]

split_data = result["split_data"]

# 建立 train_dataset 子資料夾
train_dataset_dir = os.path.join(model_dir, "train_dataset")
os.makedirs(train_dataset_dir, exist_ok=True)

# 存 parquet 到 train_dataset 資料夾
for label, df in split_data.items():
    path = os.path.join(train_dataset_dir, f"{label}.parquet")
    print(f"💾 儲存 {label} 到 {path}")
    df.write_parquet(path)


# 清理記憶體
split_data.clear()
del result
gc.collect()

print("✅ split_data 已清空，記憶體應已釋放")

all_groups_val_np = []
all_y_val_np = []
all_val_preds = []
hitrate_per_group = {}

for label in split_labels:
    print(f"\n🚀 處理分群: {label}")

    data = pl.read_parquet(os.path.join(train_dataset_dir, f"{label}.parquet"))

    split_result = prepare_train_val_split(
        result={"split_data": {label: data}},
        split_label=label,
        feature_cols=label_features[label],
        train_fraction=0.8
    )

    dtrain = xgb.DMatrix(
        split_result["X_train_np"],
        label=split_result["y_train_np"]
    )
    dtrain.set_group(split_result["group_sizes_train"])

    dval = xgb.DMatrix(
        split_result["X_val_np"],
        label=split_result["y_val_np"]
    )
    dval.set_group(split_result["group_sizes_val"])

    xgb_model = xgb.train(
        xgb_params,
        dtrain,
        num_boost_round=500,
        evals=[(dtrain, "train"), (dval, "val")],
        early_stopping_rounds=50,
        verbose_eval=20
    )

    model_path = os.path.join(model_dir, f"xgb_ranker_{label}.bin")
    xgb_model.save_model(model_path)
    print(f"✅ 已儲存模型: {model_path}")

    val_preds = xgb_model.predict(dval)

    hitrate = compute_hitrate_at_3(
        split_result["groups_val_np"],
        split_result["y_val_np"],
        val_preds
    )
    hitrate_per_group[label] = hitrate

    all_groups_val_np.append(split_result["groups_val_np"])
    all_y_val_np.append(split_result["y_val_np"])
    all_val_preds.append(val_preds)

    del dtrain, dval, xgb_model, val_preds, split_result, data
    gc.collect()
    print("🧹 記憶體已清理")

print("\n⚡ 合併所有驗證結果計算 HitRate ⚡")
all_groups_val_np = np.concatenate(all_groups_val_np)
all_y_val_np = np.concatenate(all_y_val_np)
all_val_preds = np.concatenate(all_val_preds)

overall_hitrate = compute_hitrate_at_3(
    all_groups_val_np,
    all_y_val_np,
    all_val_preds
)

print("\n💡 每組 Hitrate：")
for label, h in hitrate_per_group.items():
    print(f"{label}: {h:.4f}")

print(f"\n🎯 全部合併 HitRate: {overall_hitrate:.4f}")

# 將 hitrate 寫入 CSV
hitrate_records = [
    {"split_label": label, "hitrate": h}
    for label, h in hitrate_per_group.items()
]
hitrate_records.append({"split_label": "overall", "hitrate": overall_hitrate})

hitrate_df = pl.DataFrame(hitrate_records)
csv_path = os.path.join(model_dir, "hitrate_summary.csv")
hitrate_df.write_csv(csv_path)
print(f"\n✅ 已儲存所有 Hitrate 結果至 {csv_path}")


✅ Model folder: model_output/selected_features_xgb/v2_top20/
💾 儲存 small 到 model_output/selected_features_xgb/v2_top20/train_dataset/small.parquet
💾 儲存 medium 到 model_output/selected_features_xgb/v2_top20/train_dataset/medium.parquet
💾 儲存 large 到 model_output/selected_features_xgb/v2_top20/train_dataset/large.parquet
✅ split_data 已清空，記憶體應已釋放

🚀 處理分群: small
✅ Train: 333058 rows, 25030 groups
✅ Val: 83808 rows, 6258 groups
[0]	train-ndcg@3:0.67297	val-ndcg@3:0.66570
[20]	train-ndcg@3:0.77250	val-ndcg@3:0.73017
[40]	train-ndcg@3:0.78823	val-ndcg@3:0.73522
[60]	train-ndcg@3:0.79962	val-ndcg@3:0.73904
[80]	train-ndcg@3:0.80932	val-ndcg@3:0.74334
[100]	train-ndcg@3:0.81926	val-ndcg@3:0.74608
[120]	train-ndcg@3:0.82810	val-ndcg@3:0.74955
[140]	train-ndcg@3:0.83695	val-ndcg@3:0.75290
[160]	train-ndcg@3:0.84280	val-ndcg@3:0.75381
[180]	train-ndcg@3:0.85067	val-ndcg@3:0.75518
[200]	train-ndcg@3:0.85743	val-ndcg@3:0.75695
[220]	train-ndcg@3:0.86320	val-ndcg@3:0.75742
[240]	train-ndcg@3:0.86858	val

In [22]:
import os
import xgboost as xgb
import pandas as pd

def export_xgb_feature_importance(
    model_dir: str,
    split_labels: list,
    label_features: dict
    ):
    """
    讀取多個分群的xgb模型，輸出完整feature importance CSV。

    參數:
    - model_dir: 模型資料夾
    - split_labels: 分群label列表 (ex: ["small", "medium", ...])
    - label_features: dict(label -> 該分群使用的特徵list)
    - top_n: 顯示/列印前N重要特徵

    每個分群會輸出一個CSV在 model_importance 子資料夾。
    """
    model_importance_dir = os.path.join(model_dir, "model_importance")
    os.makedirs(model_importance_dir, exist_ok=True)

    for label in split_labels:
        features_this_label = label_features[label]
        print(f"\n🚀 處理分群 {label}... (特徵數: {len(features_this_label)})")

        model_path = os.path.join(model_dir, f"xgb_ranker_{label}.bin")
        booster = xgb.Booster()
        booster.load_model(model_path)
        print(f"✅ 已讀取模型: {model_path}")

        importance_types = ["weight", "gain", "cover"]
        importance_all = {}

        # 取得三種重要性
        for imp_type in importance_types:
            imp_raw = booster.get_score(importance_type=imp_type)
            imp_named = {}
            for k, v in imp_raw.items():
                idx = int(k[1:])
                real_name = features_this_label[idx]
                imp_named[real_name] = v
            sorted_imp = sorted(imp_named.items(), key=lambda x: x[1], reverse=True)
            importance_all[imp_type] = sorted_imp

        # 各別DataFrame
        df_weight = pd.DataFrame(importance_all["weight"], columns=["feature", "weight"])
        df_weight["weight_rank_pos"] = df_weight.index

        df_gain = pd.DataFrame(importance_all["gain"], columns=["feature", "gain"])
        df_gain["gain_rank_pos"] = df_gain.index

        df_cover = pd.DataFrame(importance_all["cover"], columns=["feature", "cover"])
        df_cover["cover_rank_pos"] = df_cover.index

        # 建完整feature表
        df_all = pd.DataFrame({"feature": features_this_label})

        # 合併
        df_merged = (
            df_all
            .merge(df_weight, on="feature", how="left")
            .merge(df_gain, on="feature", how="left")
            .merge(df_cover, on="feature", how="left")
        )

        # 補rank_pos
        df_merged["weight_rank_pos"] = df_merged["weight_rank_pos"].fillna(9999)
        df_merged["gain_rank_pos"] = df_merged["gain_rank_pos"].fillna(9999)
        df_merged["cover_rank_pos"] = df_merged["cover_rank_pos"].fillna(9999)

        # 計算最小rank
        df_merged["min_rank"] = df_merged[["weight_rank_pos", "gain_rank_pos", "cover_rank_pos"]].min(axis=1)

        # 排序
        df_merged_sorted = df_merged.sort_values("min_rank")

        # 輸出
        csv_path = os.path.join(model_importance_dir, f"feature_importance_{label}_all_features.csv")
        df_merged_sorted.to_csv(csv_path, index=False)
        print(f"✅ 已輸出特徵重要性到 {csv_path}")


In [24]:
export_xgb_feature_importance(
    model_dir=model_dir,
    split_labels=split_labels,
    label_features=label_features,  # 請用訓練用的特徵
)



🚀 處理分群 small... (特徵數: 95)
✅ 已讀取模型: model_output/selected_features_xgb/v1/xgb_ranker_small.bin
✅ 已輸出特徵重要性到 model_output/selected_features_xgb/v1/model_importance/feature_importance_small_all_features.csv

🚀 處理分群 medium... (特徵數: 99)
✅ 已讀取模型: model_output/selected_features_xgb/v1/xgb_ranker_medium.bin
✅ 已輸出特徵重要性到 model_output/selected_features_xgb/v1/model_importance/feature_importance_medium_all_features.csv

🚀 處理分群 large... (特徵數: 95)
✅ 已讀取模型: model_output/selected_features_xgb/v1/xgb_ranker_large.bin
✅ 已輸出特徵重要性到 model_output/selected_features_xgb/v1/model_importance/feature_importance_large_all_features.csv

🚀 處理分群 xlarge... (特徵數: 78)
✅ 已讀取模型: model_output/selected_features_xgb/v1/xgb_ranker_xlarge.bin
✅ 已輸出特徵重要性到 model_output/selected_features_xgb/v1/model_importance/feature_importance_xlarge_all_features.csv


In [ ]:
n_rows global_row_nr

In [1]:
import polars as pl
import pandas as pd

# 讀取 parquet
train_filled = pl.read_parquet("data/train_filled.parquet")

# 讀取top50 features


df = pd.read_csv("model_output/all_features_xgb/model_importance/common_top50_features_with_ranks.csv")
top50_features = df["feature"].head(60).tolist()

# 你要保留的欄位 (Top50 + target + group id)
cols_to_keep = top50_features + ["selected", "ranker_id"]

# 只保留這些欄位
train_filled = train_filled.select(cols_to_keep)

print(f"✅ DataFrame 現在只有 {len(train_filled.columns)} 欄位: {train_filled.columns}")


✅ DataFrame 現在只有 62 欄位: ['total_num_transfers_rank', 'total_is_min_transfers', 'price_from_median_zscore', 'price_percentile', 'legs0_segments0_flightNumber', 'price_per_duration', 'price_per_duration_rank', 'legs0_mean_cabin', 'companyID', 'legs0_max_duration_cabin', 'legs0_segments0_cabinClass', 'days_before_departure', 'legs0_weighted_mean_cabin', 'total_weighted_mean_cabin', 'price_per_fee', 'totalPrice_rank', 'legs0_arrivalAt_hour', 'legs0_departureAt_hour', 'legs1_weighted_mean_cabin', 'price_per_fee_rank', 'price_minus_fee_rank', 'price_per_tax', 'legs1_max_duration_cabin', 'has_access_tp', 'is_cheapest', 'miniRules1_monetaryAmount', 'pricingInfo_isAccessTP', 'total_duration_rank', 'log_price', 'is_popular_route', 'price_minus_fee', 'baggage_total_rank', 'legs0_segments0_seatsAvailable', 'nationality', 'isVip', 'legs0_segments0_baggageAllowance_quantity', 'total_fees', 'taxes', 'has_fees', 'miniRules0_statusInfos', 'legs1_segments0_flightNumber', 'total_duration', 'unmatched_dur

In [ ]:

import polars as pl

# 第一步：計算每個ranker_id的出現次數
group_counts = (
    train_filled
    .group_by("ranker_id")
    .agg(pl.count().alias("n_rows"))
)

# 第二步：挑選 >=10 的 ranker_id
valid_ranker_ids = (
    group_counts
    .filter(pl.col("n_rows") >= 10)
    .select("ranker_id")
)

# 第三步：過濾掉 <10 的
train_filled = (
    train_filled
    .join(valid_ranker_ids, on="ranker_id", how="inner")
)

print(f"刪掉後還有 {train_filled.height} 行資料")


刪掉後還有 18075751 行資料


In [2]:
exclude_cols = [
    'Id', 'ranker_id', 'selected'
]

feature_cols = [col for col in train_filled.columns if col not in exclude_cols]

print(f"Using {len(feature_cols)} features")

X = train_filled.select(feature_cols)
y = train_filled.select('selected')
groups = train_filled.select('ranker_id')

Using 60 features


In [ ]:
import numpy as np
import polars as pl

# 確認這些物件都是Polars DataFrame
# X, y, groups
# 都是 shape [n_rows, n_cols]

# 先把 ranker_id轉list
unique_rankers = groups.select("ranker_id").unique().to_series().to_list()

# 打亂
np.random.seed(42)
np.random.shuffle(unique_rankers)

# 切8:2
n_train = int(0.8 * len(unique_rankers))
train_rankers = set(unique_rankers[:n_train])
val_rankers = set(unique_rankers[n_train:])

# 用 Polars 過濾 train/val
is_train = groups.select(pl.col("ranker_id").is_in(list(train_rankers)).alias("is_train"))

# 先 concat mask
X_with_mask = X.with_columns(is_train)
y_with_mask = y.with_columns(is_train)
groups_with_mask = groups.with_columns(is_train)

# 分割 DataFrame
X_train_df = X_with_mask.filter(pl.col("is_train"))
X_val_df = X_with_mask.filter(~pl.col("is_train"))
y_train_df = y_with_mask.filter(pl.col("is_train"))
y_val_df = y_with_mask.filter(~pl.col("is_train"))
groups_train_df = groups_with_mask.filter(pl.col("is_train"))
groups_val_df = groups_with_mask.filter(~pl.col("is_train"))

# 再轉 numpy (分批)
X_train_np = X_train_df.drop("is_train").to_numpy()
X_val_np = X_val_df.drop("is_train").to_numpy()
y_train_np = y_train_df.drop("is_train").to_numpy().flatten()
y_val_np = y_val_df.drop("is_train").to_numpy().flatten()
groups_train_np = groups_train_df.drop("is_train").to_numpy().flatten()
groups_val_np = groups_val_df.drop("is_train").to_numpy().flatten()
# 最後計算 group sizes
group_sizes_train = (
    groups_train_df.drop("is_train")
    .group_by("ranker_id", maintain_order=True)
    .agg(pl.len())['len']
    .to_numpy()
)

group_sizes_val = (
    groups_val_df.drop("is_train")
    .group_by("ranker_id", maintain_order=True)
    .agg(pl.len())['len']
    .to_numpy()
)


In [4]:
import xgboost as xgb

# DMatrix 建立 (不再用 X.columns)
dtrain = xgb.DMatrix(
    X_train_np,
    label=y_train_np,
)
dtrain.set_group(group_sizes_train)

dval = xgb.DMatrix(
    X_val_np,
    label=y_val_np,
)
dval.set_group(group_sizes_val)


In [7]:

# 參數
xgb_params = {
    'objective': 'rank:pairwise',
    'eval_metric': 'ndcg@3',
    'max_depth': 10,
    'min_child_weight': 10,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'lambda': 10.0,
    'learning_rate': 0.05,
    'seed': 42,
    'n_jobs': -1,
}

# 訓練
xgb_model = xgb.train(
    xgb_params,
    dtrain,
    num_boost_round=500,
    evals=[(dtrain, "train"), (dval, "val")],
    early_stopping_rounds=50,
    verbose_eval=20
)
# 存成二進位模型
xgb_model.save_model("xgb_ranker_model.bin")
print("✅ 已儲存模型到 'xgb_ranker_model.bin'")



[0]	train-ndcg@3:0.79231	val-ndcg@3:0.78756
[20]	train-ndcg@3:0.82876	val-ndcg@3:0.81002
[40]	train-ndcg@3:0.83681	val-ndcg@3:0.81411
[60]	train-ndcg@3:0.84316	val-ndcg@3:0.81633
[80]	train-ndcg@3:0.84911	val-ndcg@3:0.81838
[100]	train-ndcg@3:0.85485	val-ndcg@3:0.82003
[120]	train-ndcg@3:0.85969	val-ndcg@3:0.82135
[140]	train-ndcg@3:0.86504	val-ndcg@3:0.82257
[160]	train-ndcg@3:0.87028	val-ndcg@3:0.82400
[180]	train-ndcg@3:0.87456	val-ndcg@3:0.82537
[200]	train-ndcg@3:0.87932	val-ndcg@3:0.82704
[220]	train-ndcg@3:0.88309	val-ndcg@3:0.82761
[240]	train-ndcg@3:0.88689	val-ndcg@3:0.82810
[260]	train-ndcg@3:0.89009	val-ndcg@3:0.82924
[280]	train-ndcg@3:0.89300	val-ndcg@3:0.82984
[300]	train-ndcg@3:0.89572	val-ndcg@3:0.83076
[320]	train-ndcg@3:0.89829	val-ndcg@3:0.83126
[340]	train-ndcg@3:0.90079	val-ndcg@3:0.83153
[360]	train-ndcg@3:0.90291	val-ndcg@3:0.83223
[380]	train-ndcg@3:0.90545	val-ndcg@3:0.83341
[400]	train-ndcg@3:0.90730	val-ndcg@3:0.83396
[420]	train-ndcg@3:0.90960	val-ndcg@3:0.

/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_48131/475441800.py:25: UserWarning: [00:24:15] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1748292887431/work/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model("xgb_ranker_model.bin")


In [8]:
import polars as pl
from scripts.hitrate import compute_hitrate_at_3
# 預測
val_preds = xgb_model.predict(dval)

# 計算 HitRate
compute_hitrate_at_3(groups_val_np, y_val_np, val_preds)


✅ HitRate@3 (groups size in [10, inf]): 0.5122


0.5121602288984263

In [6]:
import polars as pl
from scripts.hitrate import compute_hitrate_at_3
# 預測
val_preds = xgb_model.predict(dval)

# 計算 HitRate
compute_hitrate_at_3(groups_val_np, y_val_np, val_preds)


✅ HitRate@3 (groups size in [10, inf]): 0.4488


0.4487729723781226

In [10]:
y_val_np

array([0, 1, 0, ..., 0, 0, 0], shape=(3769685,))

In [12]:
# 存成二進位模型
xgb_model.save_model("xgb_ranker_model.bin")
print("✅ 已儲存模型到 'xgb_ranker_model.bin'")


✅ 已儲存模型到 'xgb_ranker_model.bin'


In [19]:
# 取得重要性 (index key: f0, f1, ...)
importance_dict = xgb_model.get_score(importance_type="weight")

# 把特徵名稱對應回 X.columns
# 如果X是 Polars DataFrame
feature_names = list(X.columns)

# 重新命名
importance_named = {}
for k, v in importance_dict.items():
    # k = 'f0', 'f1', ...
    idx = int(k[1:])  # 取數字
    real_name = feature_names[idx]
    importance_named[real_name] = v

# 排序
sorted_importance = sorted(importance_named.items(), key=lambda x: x[1], reverse=True)

# 列印
print("⭐ Top 20 Feature Importances (by weight):")
for feat, score in sorted_importance[:20]:
    print(f"{feat}: {score}")


⭐ Top 20 Feature Importances (by weight):
totalPrice: 8578.0
companyID: 6165.0
legs0_segments0_flightNumber: 6092.0
legs1_segments0_flightNumber: 5554.0
days_before_departure: 5454.0
legs1_segments0_seatsAvailable: 3737.0
legs0_segments0_seatsAvailable: 3721.0
legs0_arrivalAt_hour: 3577.0
legs1_arrivalAt_hour: 3280.0
legs0_departureAt_hour: 3152.0
legs1_departureAt_hour: 2996.0
corporateTariffCode: 2835.0
taxes: 2525.0
legs1_departureAt_weekday: 1990.0
legs0_departureAt_weekday: 1817.0
legs0_arrivalAt_weekday: 1588.0
legs1_arrivalAt_weekday: 1501.0
frequentFlyer: 1446.0
legs1_duration: 1376.0
legs0_duration: 1354.0


In [17]:
import pandas as pd

# 先準備特徵名稱
feature_names = list(X.columns)

# 取三種重要性
importance_types = ["weight", "gain", "cover"]
importance_all = {}

for imp_type in importance_types:
    imp_raw = xgb_model.get_score(importance_type=imp_type)
    imp_named = {}
    for k, v in imp_raw.items():
        idx = int(k[1:])
        real_name = feature_names[idx]
        imp_named[real_name] = v
    # 排序
    sorted_imp = sorted(imp_named.items(), key=lambda x: x[1], reverse=True)
    importance_all[imp_type] = sorted_imp

# 把三個榜單放成DataFrame方便比對
df_weight = pd.DataFrame(importance_all["weight"], columns=["feature", "weight_rank"])
df_weight["weight_rank_pos"] = df_weight.index

df_gain = pd.DataFrame(importance_all["gain"], columns=["feature", "gain_rank"])
df_gain["gain_rank_pos"] = df_gain.index

df_cover = pd.DataFrame(importance_all["cover"], columns=["feature", "cover_rank"])
df_cover["cover_rank_pos"] = df_cover.index

# 合併
df_merged = (
    df_weight
    .merge(df_gain, on="feature", how="outer")
    .merge(df_cover, on="feature", how="outer")
)

# 把不存在的rank補大數字
df_merged["weight_rank_pos"] = df_merged["weight_rank_pos"].fillna(9999)
df_merged["gain_rank_pos"] = df_merged["gain_rank_pos"].fillna(9999)
df_merged["cover_rank_pos"] = df_merged["cover_rank_pos"].fillna(9999)

# 計算「三個榜單中最早出現的位置」
df_merged["min_rank"] = df_merged[["weight_rank_pos", "gain_rank_pos", "cover_rank_pos"]].min(axis=1)

# 排序
df_merged_sorted = df_merged.sort_values("min_rank")

# 取前50
top50 = df_merged_sorted.head(50)

# 顯示
print(top50[["feature", "weight_rank", "gain_rank", "cover_rank"]])

# 如果想輸出CSV
top50.to_csv("feature_importance_top50.csv", index=False)
print("✅ 已輸出 feature_importance_top50.csv")


                                        feature  weight_rank  gain_rank  \
76                                   totalPrice       8578.0   4.235002   
22                   legs0_segments0_cabinClass        109.0  42.103062   
1                                     companyID       6165.0   3.913192   
34   legs0_segments1_departureFrom_airport_iata          1.0  41.605595   
26        legs0_segments0_marketingCarrier_code        153.0   3.633880   
62   legs1_segments1_departureFrom_airport_iata          2.0   7.080189   
25                 legs0_segments0_flightNumber       6092.0   4.140559   
0                              both_legs_direct         35.0  21.166515   
60  legs1_segments1_arrivalTo_airport_city_iata          1.0  18.557640   
55                 legs1_segments0_flightNumber       5554.0   4.186655   
16                           legs0_main_carrier         22.0   3.867804   
3                         days_before_departure       5454.0   3.778232   
20    legs0_segments0_bag

In [ ]:

data_xgb = X.with_columns([(pl.col(c).rank("dense") - 1).fill_null(-1).cast(pl.Int32) for c in cat_features_final])

n1 = 16487352 # split train to train and val (10%) in time
n2 = train.height
data_xgb_tr, data_xgb_va, data_xgb_te = data_xgb[:n1], data_xgb[n1:n2], data_xgb[n2:]
y_tr, y_va, y_te = y[:n1], y[n1:n2], y[n2:]
groups_tr, groups_va, groups_te = groups[:n1], groups[n1:n2], groups[n2:]

group_sizes_tr = groups_tr.group_by('ranker_id').agg(pl.len()).sort('ranker_id')['len'].to_numpy()
group_sizes_va = groups_va.group_by('ranker_id').agg(pl.len()).sort('ranker_id')['len'].to_numpy()
group_sizes_te = groups_te.group_by('ranker_id').agg(pl.len()).sort('ranker_id')['len'].to_numpy()
dtrain = xgb.DMatrix(data_xgb_tr, label=y_tr, group=group_sizes_tr, feature_names=data_xgb.columns)
dval   = xgb.DMatrix(data_xgb_va, label=y_va, group=group_sizes_va, feature_names=data_xgb.columns)
dtest  = xgb.DMatrix(data_xgb_te, label=y_te, group=group_sizes_te, feature_names=data_xgb.columns)

Id,ranker_id,selected,profileId,requestDate,miniRules0_percentage,miniRules1_percentage,frequentFlyer,pricingInfo_passengerCount
i64,str,i64,i64,datetime[ns],f64,f64,i32,i64
0,"""98ce0dabf6964640b63079fbafd42c…",1,2087645,2024-05-17 03:03:08,0.0,0.0,95,1
1,"""98ce0dabf6964640b63079fbafd42c…",0,2087645,2024-05-17 03:03:08,0.0,0.0,95,1
2,"""98ce0dabf6964640b63079fbafd42c…",0,2087645,2024-05-17 03:03:08,0.0,0.0,95,1
3,"""98ce0dabf6964640b63079fbafd42c…",0,2087645,2024-05-17 03:03:08,0.0,0.0,95,1
4,"""98ce0dabf6964640b63079fbafd42c…",0,2087645,2024-05-17 03:03:08,0.0,0.0,95,1
…,…,…,…,…,…,…,…,…
18146427,"""88f8c53a28bf4f438941fd67338009…",0,3046852,2024-10-29 12:46:20,0.0,0.0,371,1
18146428,"""88f8c53a28bf4f438941fd67338009…",0,3046852,2024-10-29 12:46:20,0.0,0.0,371,1
18146429,"""88f8c53a28bf4f438941fd67338009…",0,3046852,2024-10-29 12:46:20,0.0,0.0,371,1
